In [ ]:
# default_exp core

In [ ]:
#export
from medtop.nlp_helpers import *
from medtop.preprocessing import *

# Core

> Methods for calling 

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def import_docs(path_to_file_list, verbose = False):
    "Imports and processes the list of documents contained in the given file"
    
    # Extract list of files from text document
    with open(path_to_file_list) as file:
        file_list = file.read().strip().split('\n')
    
    # Append the raw contents of each document to an array
    file_content = []
    for file_path in file_list:
        with open(file_path, "r") as file:
            text = file.read()
            file_content.append(text)
        
    my_docs = []
    my_docs_pos = []
    my_docs_loc = []    
    raw_sentences = []

    for doc in file_content:
        split_sent, split_sent_pos, split_sent_loc, tokens, pos, loc, full_sent, full_pos, full_loc, raw_sent = tokenize_and_stem(doc)
        my_docs.append(split_sent)
        my_docs_pos.append(split_sent_pos)
        my_docs_loc.append(split_sent_loc)
        
        raw_sentences.append(raw_sent)
        
    if verbose == True: print("Number of Documents Loaded: " + str(len(my_docs)))
        
    return my_docs, my_docs_pos, my_docs_loc, raw_sentences

In [ ]:
#export
def sentences_to_disk(raw_sentences, outfile_name):
    "Writes the raw sentences to a file organized by document and sentence number"
#     outfile_name = args.o + "/" + args.p + "_DocumentSentenceList.txt"
    with open(outfile_name, "w") as outfile:
        outfile.write("Sentence ID\tSentence Text\n")
        for doc_ix, doc in enumerate(raw_sentences):
            for sent_ix, sent in enumerate(doc):
                outfile.write(f"doc.{doc_ix}.sent.{sent_ix}\t{sent}\n")

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted Core.ipynb.
Converted helpers.ipynb.
Converted index.ipynb.
Converted nlp_helpers.ipynb.
Converted preprocessing.ipynb.
Converted Sandbox.ipynb.
